<a href="https://colab.research.google.com/github/phuocnguyen90/Random-projects/blob/main/WebUI_vi_gemma_2b_RAG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please make sure you run this notebook with T4 runtime (change runtime type)

Hãy chắc rằng bạn đang chạy notebook này với môi trường T4

In [ ]:
# disable cell output
%%capture
# update relevant packages
!pip install transformers torch accelerate
!pip install --upgrade gradio

In [ ]:
# disable cell output
%%capture
import warnings
warnings.filterwarnings('ignore')
# import relvant packages
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Khởi tạo tokenizer và model từ checkpoint đã lưu
# Cell này sẽ mất khoảng 5-6 phút để tải và load mô hình từ huggingface
tokenizer = AutoTokenizer.from_pretrained("himmeow/vi-gemma-2b-RAG")
model = AutoModelForCausalLM.from_pretrained(
    "himmeow/vi-gemma-2b-RAG",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Sử dụng GPU nếu có
if torch.cuda.is_available():
    model.to("cuda")

from transformers import pipeline



In [ ]:
vi_gemma = pipeline(
    "document-question-answering",  # RAG task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
# Định dạng prompt cho model
PROMPT = """
### Instruction and Input:
Dựa vào ngữ cảnh/tài liệu sau:
{}
Hãy trả lời câu hỏi: {}

### Response:
{}
"""

# Chuẩn bị dữ liệu đầu vào
INPUT_DATA = """
3. Quy định về lập phương án và dự toán chi phí
3.1. Nguyên tắc
- Tất cả các khoản chi đều phải được phê duyệt chủ trương trước khi thực hiện.
- Tất cả các khoản chi phải lập dự toán chi phí, kể cả chi phí lập, thẩm định, đánh giá hồ sơ, chi phí dự phòng (chi phí dự phòng trượt giá, chi phí dự phòng phát sinh khối lượng), phí, lệ phí và thuế.
- Tất cả các khoản chi đều phải lập phương án thực hiện. Phương án ở đây có thể hiểu là phương án ngắn gọn (nội dung tối thiểu ở mục 3.3.1) được thể hiện trong cùng nội dung tờ trình chủ trương, hoặc phương án thực hiện thể hiện các nội dung thực hiện từng bước, được trình bày và phê duyệt trong một văn bản riêng, tùy theo mức độ phức tạp của việc thực hiện khoản chi.
- Trên cơ sở phương án đã được phê duyệt, Đơn vị thực hiện tiến hành tìm kiếm hoặc lựa chọn nhà thầu/nhà cung cấp để thực hiện phương án theo quy trình được quy định tại Quy định này hoặc theo quy định của Luật đấu thầu.
3.2. Đơn vị lập
- Đơn vị thực hiện sẽ chủ trì lập phương án. Phương án cần được thẩm tra hoặc thẩm định trước khi trình cho Người có thẩm quyền phê duyệt phương án.
- Các phương án phức tạp, nếu đơn vị thực hiện không đủ khả năng lập hồ sơ thì có thể thuê đơn vị tư vấn có đủ điều kiện, năng lực phù hợp lập phương án.
3.3. Quy định về lập phương án và dự toán chi phí
3.3.1. Đối với các phương án với hạn mức đến 100 triệu đồng
Nội dung chủ trương bao gồm: Tên phương án/công việc; Căn cứ pháp lý; Đánh giá mức độ cần thiết; Các nội dung cần thực hiện; Thời gian thực hiện.
Nội dung phương án tối thiểu phải bao gồm: Yêu cầu thực tế của việc xây dựng phương án; Các nội dung chi tiết cần thực hiện; Dự toán chi phí cho việc thực hiện phương án; Kế hoạch lựa chọn nhà thầu/nhà cung cấp; Hình thức quản lý và thực hiện.
Căn cứ vào mức độ phức tạp của dự án, Người thẩm quyền có thể yêu cầu làm rõ thêm các nội dung phương án như liệt kê ở mục 3.3.2.
3.3.2. Đối với các phương án với hạn mức trên 100 triệu đồng
Nội dung của phương án bao gồm:
- Sự cần thiết của phương án.
- Mục tiêu.
- Địa điểm trang bị/thực hiện/cải tạo.
- Quy mô phương án, phạm vi cung cấp bao gồm chủng loại hàng hóa/loại hình dịch vụ và số lượng cung cấp.
- Nguồn vốn
- Dự toán chi phí thực hiện.
- Tiêu chí kỹ thuật chi tiết.
- Thời gian thực hiện hợp đồng, tiến độ cung cấp, triển khai.
- Mẫu hàng hóa, sản phẩm (nếu có).
- Tiến độ thực hiện.
- Hiệu quả khi thực hiện phương án.
- Kế hoạch lựa chọn nhà thầu/nhà cung cấp.
- Các điều kiện cần thiết khác.
- Hồ sơ phương án phải đóng thành quyển, có người lập và Trưởng đơn vị ký, đóng dấu (nếu có).
3.3.3. Lưu ý: Trong phương án không nêu xuất xứ, nhãn hiệu của hàng hóa, trong trường hợp đặc biệt cần thiết phải nêu xuất xứ, nhãn hiệu của một nhà sản xuất nào đó, hoặc hàng hóa từ một nước nào đó để tham khảo, minh họa cho yêu cầu kỹ thuật của hàng hóa thì phải kèm theo cụm từ “hoặc tương đương” sau nhãn hiệu hoặc xuất xứ nêu ra và quy định rõ khái niệm tương đương nghĩa là có đặc tính kỹ thuật tương tự, có tính năng sử dụng là tương đương với các hàng hóa đã nêu.
3.4. Căn cứ để lập dự toán
- Quyết định giao kế hoạch chi phí sản xuất kinh doanh hàng năm/quý.
- Giá dự toán chi phí: Thực hiện theo quy định tại Điều 16 Nghị định 24/2024/NĐ-CP. Một số điểm cần lưu ý khi thực hiện dự toán chi phí:
 Nếu công việc dự định thực hiện có định mức, đơn giá do Nhà nước công bố thì giá dự toán chi phí được hình thành trên cơ sở các đơn giá đó. Dự toán có thể được lập trên cơ sở đơn giá của kết quả đấu thầu/chào hàng cạnh tranh/chào hàng cạnh tranh gần nhất hoặc hợp đồng tương tự đã thực hiện trước đó.
 Nếu công việc dự định thực hiện đã có đơn giá trần do Tổng công ty quy định và ban hành thì giá dự toán chi phí được hình thành trên cơ sở không vượt đơn giá trần và có lợi nhất cho Công ty, Tổng công ty.
 Nếu công việc dự định thực hiện không có định mức, đơn giá do Nhà nước ban hành, không có đơn giá trần do Tổng công ty quy định thì giá dự toán chi phí phải được hình thành từ việc lập chi tiết từng nội dung kết hợp nghiên cứu khảo sát thực tế thị trường trên cơ sở tuân thủ theo quy định tại điểm b khoản 3 Điều 15, các điểm d, e, g khoản 2 Điều 16 Nghị định 24/2024/NĐ-CP. Theo đó, việc khảo sát giá cơ sở thông qua tối thiểu 01 báo giá của hàng hóa, dịch vụ, nhưng khuyến khích thu thập nhiều hơn 01 báo giá để có sự so sánh; trường hợp có nhiều hơn 01 báo giá thì lấy giá trung bình của các báo giá. Trong trường hợp không có nhiều nhà cung cấp trên địa bàn để so sánh thì có thể tham khảo trên địa bàn khác; hoặc từ các nguồn thông tin do cơ quan có thẩm quyền công bố, hoặc theo giá niêm yết của nhà sản xuất, nhà nhập khẩu, đại lý, nhà phân phối, nhà cung ứng, doanh nghiệp…
 Trường hợp chỉ định thầu, chỉ định nhà cung cấp: nếu công việc lập dự toán không có các định mức, đơn giá của Nhà nước, Tổng công ty ban hành thì việc lập dự toán dựa trên cơ sở:
 Kết quả đấu thầu, chào hàng cạnh tranh, hợp đồng tương tự gần nhất trước đó.
 Yêu cầu nhà cung cấp/nhà thầu được đề nghị chỉ định thực hiện phương án giải trình phương án xây dựng chi phí hợp lý đồng thời cung cấp hợp đồng tương tự đã thực hiện (nếu có).
 Nếu công việc phát sinh lần đầu (chưa có hợp đồng tương tự) thì đơn vị trình, đơn vị thẩm định và người có thẩm quyền phê duyệt phương án phải xem xét tính hợp lý của giá mà nhà cung cấp/nhà thầu được đề nghị chỉ định thực hiện phương án đưa ra và chịu trách nhiệm về quyết định của mình.
 Trường hợp gia hạn hợp đồng: dự toán, đơn giá không vượt hợp đồng đang thực hiện, tham khảo đơn giá các hợp đồng tương tự hoặc kết quả đấu thầu/chào hàng cạnh tranh gần nhất để đàm phán áp dụng đơn giá thấp nhất.
 Trường hợp mua sắm trực tiếp: căn cứ kết quả đấu thầu trước đó không quá 12 tháng kể từ ngày ký hợp đồng trước đó.
3.5. Yêu cầu chung về tính hợp lệ của báo giá
- Các báo giá đều phải được Người có thẩm quyền của đơn vị báo giá ký tên bằng chữ ký tươi, có đóng dấu và báo giá phải còn thời hạn hiệu lực.
- Người trình phương án phải chịu trách nhiệm về tính pháp nhân, xác thực của các đơn vị cung cấp báo giá và tính hợp lý của báo giá, phương án trình phê duyệt.
3.6. Kế hoạch lựa chọn nhà thầu/nhà cung cấp
- Kế hoạch lựa chọn nhà thầu/nhà cung cấp được lập cho toàn bộ dự toán mua sắm hoặc một số gói thầu/gói mua sắm thuộc dự toán mua sắm để thực hiện trước. Trong kế hoạch lựa chọn nhà thầu/nhà cung cấp phải ghi rõ số lượng gói thầu/gói mua sắm và nội dung của từng gói thầu/gói mua sắm và việc áp dụng thỏa thuận khung (nếu có).
- Việc phân chia dự toán mua sắm thành các gói thầu/gói mua sắm phải căn cứ theo tính chất kỹ thuật, trình tự thực hiện; đảm bảo tính đồng bộ và quy mô gói thầu/gói mua sắm hợp lý; không được phép chia nhỏ để áp dụng hình thức lựa chọn kém cạnh tranh hơn.
- Nguồn chi phí thực hiện trong kế hoạch lựa chọn nhà thầu/nhà cung cấp:
 Căn cứ Quyết định giao kế hoạch chi phí sản xuất kinh doanh hàng năm/quý.
 Đối với các khoản chi cần thực hiện đấu thầu hoặc đấu thầu tập trung để ký hợp đồng khung áp dụng cho nhiều kỳ kế toán thì khi lập phương án không nhất thiết phải có kế hoạch chi phí cho những năm sau. Nguồn chi phí/vốn thực hiện ghi: Chi phí sản xuất kinh doanh của Công ty.
- Chủ thể ký hợp đồng: Trong trường hợp Công ty đứng ra tổ chức lựa chọn nhà thầu/nhà cung cấp và đơn vị trực thuộc Công ty ký hợp đồng, chủ thể ký kết hợp đồng phải được xác định rõ trong quyết định phê duyệt phương án.
3.7. Hồ sơ trình thẩm tra, thẩm định, phê duyệt phương án và dự toán chi phí
3.6.1. Hồ sơ thẩm tra để phê duyệt phương án gồm:
- Tờ trình phê duyệt phương án
- Phương án thực hiện và dự toán chi phí.
- Các hồ sơ khác có liên quan.
Đơn vị thực hiện sẽ tiến hành tổ chức lập phương án và dự toán chi phí theo đúng các quy định hiện hành và chịu trách nhiệm trước Giám đốc Công ty và trước pháp luật về các hồ sơ này, trong trường hợp cần thuê đơn vị tư vấn bên ngoài để tiến hành các công việc trên cần phải báo cáo người có thẩm quyền xem xét chấp thuận.
3.6.2. Hồ sơ thẩm định các bước theo quy định của Luật đấu thầu thì thực hiện theo quy định của pháp luật.


"""



In [ ]:
# Câu hỏi
query = "Các căn cứ nào để xác định giá dự toán chi phí?"

In [ ]:
# Generate a response from the  model directly
def get_vi_gemma_response(query):
    input_text = PROMPT.format(INPUT_DATA, query," ")

    # Mã hóa input text thành input ids
    input_ids = tokenizer(input_text, return_tensors="pt")

    # Sử dụng GPU cho input ids nếu có
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    # Tạo văn bản bằng model
    outputs = model.generate(
        **input_ids,
        max_new_tokens=500,
        no_repeat_ngram_size=5,  # Ngăn chặn lặp lại các cụm từ 5 gram
        #do_sample=True,   # Kích hoạt chế độ tạo văn bản dựa trên lấy mẫu. Trong chế độ này, model sẽ chọn ngẫu nhiên token tiếp theo dựa trên xác suất được tính từ phân phối xác suất của các token.
        temperature=0.5,  # Giảm temperature để kiểm soát tính ngẫu nhiên
        early_stopping=True,  # Dừng tạo văn bản khi tìm thấy kết thúc phù hợp
    )
    # Giải mã và xử lý kết quả để chỉ lấy phần bắt đầu với "### Response:"
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tìm phần bắt đầu với "### Response:" và cắt phần còn lại
    response_start = decoded_output.find("### Response:")
    if response_start != -1:
        response_text = decoded_output[response_start + len("### Response:"):].strip()
    else:
        response_text = decoded_output
    return response_text


Warning, the next cell would take  A LOT of time to generate an answer if not run on T4 GPU

In [ ]:
# chạy ô này để lấy kết quả câu hỏi
response=get_vi_gemma_response(query)
print(response)


KeyboardInterrupt: 



---

# Phần này để kích hoạt giao diện người dùng

In [ ]:
# Generate a response from the  model through pipeline
# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    # Format the history part of the prompt
    context = ""
    for user_msg, model_answer in history:
        context += f"User: {user_msg}\nModel: {model_answer}\n"

    # Combine the context, current message, and prompt structure
    formatted_message = PROMPT.format(context, message, "")
    return formatted_message



In [ ]:
def get_gemma_response(message: str, input_data: str) -> str:
    # Format the input text
    input_text = PROMPT.format(input_data, message, "")

    # Encode the input text
    input_ids = tokenizer(input_text, return_tensors="pt")

    # Use GPU if available
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    # Generate text using the model
    sequence = model.generate(
        **input_ids,
        max_new_tokens=500,
        no_repeat_ngram_size=5,
        temperature=0.5,
        early_stopping=True
    )

    # Decode the output and extract the response
    decoded_output = tokenizer.decode(sequence[0], skip_special_tokens=True)
    response_start = decoded_output.find("### Response:")
    if response_start != -1:
        response_text = decoded_output[response_start + len("### Response:"):].strip()
    else:
        response_text = decoded_output

    return response_text

In [ ]:
import gradio as gr
# Gradio interface# Gradio interface
def chat_interface(message, history):
    response = get_gemma_response(message)
    history.append([message, response])  # Ensure history is a list of lists
    return history, history

# Function to handle user message
def user(message, history, input_data):
    history.append([message, None])
    return "", history, input_data

# Function to handle bot response
def chat_interface(history, input_data):
    user_message = history[-1][0]
    response = get_gemma_response(user_message, input_data)
    history[-1][1] = response
    return history

# Function to enable input field
def enable_input():
    return gr.update(interactive=True)

# Define Gradio blocks interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_data_box = gr.Textbox(label="Nội dung văn bản", placeholder="Chép nội dung văn bản cần tìm kiếm vào đây", lines=20, elem_id="input-data-box")
        with gr.Column():
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="Your Message", placeholder="Enter your message here...")
            with gr.Row():
                submit = gr.Button("Submit")
                clear = gr.Button("Clear")

    msg.submit(user, [msg, chatbot, input_data_box], [msg, chatbot, input_data_box], queue=False).then(
        chat_interface, [chatbot, input_data_box], chatbot
    ).then(
        enable_input, None, [msg]
    )
    submit.click(user, [msg, chatbot, input_data_box], [msg, chatbot, input_data_box], queue=False).then(
        chat_interface, [chatbot, input_data_box], chatbot
    ).then(
        enable_input, None, [msg]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

# Custom CSS to style the input data box
demo.css = """
#input-data-box {
    border: 2px solid #000;
    padding: 10px;
    border-radius: 5px;
    background-color: #f9f9f9;
    font-family: Arial, sans-serif;
    font-size: 14px;
}
"""

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7eda8102887bbab313.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Document chunking (work in progress)
In this section we will chunk and embed longer document with the embedding model halong_embedding to be able to do RAG within a longer document

In [ ]:
def chunk_document(document, chunk_size=300):
    """
    Splits the document into chunks of specified size.

    Parameters:
        document (str): The input document.
        chunk_size (int): The size of each chunk.

    Returns:
        list: A list of document chunks.
    """
    words = document.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]


In [ ]:
%%capture
!pip install sentence_transformers
!pip install rank_bm25

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the halong_embedding model
embedding_model = SentenceTransformer("hiieu/halong_embedding")

def rank_chunks(query):
    global context_chunks, context_embeddings
    query_embedding = embedding_model.encode([query])
    similarities = torch.tensor([embedding_model.similarity(query_embedding, [chunk_emb]).flatten() for chunk_emb in context_embeddings])
    sorted_indices = torch.argsort(similarities, descending=True).flatten()
    ranked_chunks = [context_chunks[idx] for idx in sorted_indices]
    return ranked_chunks

In [ ]:
def get_relevant_context(document, query, top_k=5):
    """
    Retrieves the most relevant context for the query from the document.

    Parameters:
        document (str): The input document.
        query (str): The user's query.
        top_k (int): The number of top relevant chunks to retrieve.

    Returns:
        str: The relevant context.
    """
    chunks = chunk_document(document)
    ranked_chunks = rank_chunks(query, chunks)
    return ' '.join(ranked_chunks[:top_k])




In [ ]:
def get_relevant_context(document, query, top_k=5):
    """
    Retrieves the most relevant context for the query from the document.

    Parameters:
        document (str): The input document.
        query (str): The user's query.
        top_k (int): The number of top relevant chunks to retrieve.

    Returns:
        str: The relevant context.
    """
    chunks = chunk_document(document)
    ranked_chunks = rank_chunks(chunks, query)
    return ' '.join(ranked_chunks[:top_k])

def chunk_and_embed_context(input_data):
    global context_chunks, context_embeddings
    context_chunks = chunk_document(input_data)
    context_embeddings = embedding_model.encode(context_chunks)
    return "Context has been chunked and embedded."


In [ ]:
chunk_and_embed_context(INPUT_DATA)

'Context has been chunked and embedded.'

In [ ]:
# Function to generate the response using the chatbot model
def get_gemma_response(message: str) -> str:
    context = get_relevant_context(message)
    input_text = PROMPT.format(context, message, "")
    input_ids = tokenizer(input_text, return_tensors="pt")
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
    sequence = model.generate(
        **input_ids,
        max_new_tokens=500,
        no_repeat_ngram_size=5,
        temperature=0.5,
        early_stopping=True
    )
    decoded_output = tokenizer.decode(sequence[0], skip_special_tokens=True)
    response_start = decoded_output.find("### Response:")
    if response_start != -1:
        response_text = decoded_output[response_start + len("### Response:"):].strip()
    else:
        response_text = decoded_output
    return response_text


In [ ]:
import gradio as gr

# Define Gradio blocks interface

# Define Gradio blocks interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_data_box = gr.Textbox(label="Context or Document", placeholder="Enter your context or document here...", lines=20, elem_id="input-data-box")
            chunk_button = gr.Button("Chunk and Embed Context")
        with gr.Column():
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="Your Message", placeholder="Enter your message here...")
            with gr.Row():
                submit = gr.Button("Submit")
                clear = gr.Button("Clear")

    chunk_button.click(chunk_and_embed_context, input_data_box, None)
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chat_interface, chatbot, chatbot
    ).then(
        enable_input, None, [msg]
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chat_interface, chatbot, chatbot
    ).then(
        enable_input, None, [msg]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

# Custom CSS to style the input data box
demo.css = """
#input-data-box {
    border: 2px solid #000;
    padding: 10px;
    border-radius: 5px;
    background-color: #f9f9f9;
    font-family: Arial, sans-serif;
    font-size: 14px;
}
"""

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://793178710292194533.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
